In [1]:
import numpy as np 
import os 
import pandas as pd 
import re
import string
import seaborn as sns
from rake_nltk import Rake
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import vstack

In [2]:
df_anime = pd.read_csv("../data/df_anime_export.csv")

## View Resetter

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
pd.reset_option('display.max_columns')
pd.reset_option('display.max_rows')
pd.reset_option('display.max_colwidth')

## Cleaning anime_df a bit more 

In [3]:
df_anime.head(1)

,Unnamed: 0,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,synopsis_processed
0,0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,"following participation inter-high , karasuno ..."


In [4]:
for i, genre in enumerate(df_anime['genre']):
    df_anime.at[i, 'genre'] = genre.replace('Martial Arts', 'Martial_Arts')
    df_anime.at[i, 'genre'] = genre.replace('Super Power', 'Super_Power')
    df_anime.at[i, 'genre'] = genre.replace('Slice of Life', 'Slice_of_Life')

In [5]:
#clean genre so it can be used in the rec system
for i, row_genre in enumerate(df_anime['genre']):
    row_genre = row_genre.replace('[','')
    row_genre = row_genre.replace(']','')
    row_genre = row_genre.replace("'","")
    row_genre = row_genre.replace(",","")
    df_anime['genre'].iloc[i] = row_genre
df_anime['genre']

F:\Anaconda\envs\learn-env\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


0                       Comedy Sports Drama School Shounen
1                       Drama Music Romance School Shounen
2                   Sci-Fi Adventure Mystery Drama Fantasy
3        Action Military Adventure Comedy Drama Magic F...
4                      Action Mystery Supernatural Vampire
                               ...                        
16209       Action Comedy Super Power Martial Arts Shounen
16210                    Slice_of_Life Comedy Supernatural
16211                         Slice_of_Life Comedy Shounen
16212                                               Action
16213            Comedy Drama Romance School Slice_of_Life
Name: genre, Length: 16214, dtype: object

## Begin recc model with preprocessed synopsis

In [12]:
syp = df_anime['synopsis_processed']
genre = df_anime['genre']
title = df_anime['title']

In [13]:
df_touse = df_anime[['synopsis_processed', 'genre','title']]

In [14]:
#extracting keywords for recommender
rake = Rake()
keywords = []
for plot in syp:
    rake.extract_keywords_from_text(str(plot))
    keywords_i = rake.get_ranked_phrases()
    keywords_i_string = ""
    for keyword in keywords_i:
        keywords_i_string = keywords_i_string + " " + keyword
    keywords.append(keywords_i_string)
df_touse['keywords'] = keywords

<ipython-input-14-d6bbce4ef9e3>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_touse['keywords'] = keywords


' volleyball team must learn settle difference train harder ever hope overcome formidable opponent old new — including archrival aoba jousai world karasuno agrees take part large training camp alongside many notable volleyball team tokyo even national level player karasuno high school volleyball team attempt refocus effort facing may last chance victory senior player graduate also come new attack would strengthen hinata kageyama attempt devise powerful weapon one could possibly break sturdiest block aiming conquer spring tournament instead playing toughest team japan hope sharpen skill standing rival nekoma high class setter tooru oikawa member karasuno receive invitation long following participation inter high moreover'

In [15]:
df_touse['bag_words'] = df_touse['genre'] + df_touse['keywords']

<ipython-input-15-3a53a76f0bbb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_touse['bag_words'] = df_touse['genre'] + df_touse['keywords']


In [16]:
vectorizer = CountVectorizer()
vectorized_bag_of_words = vectorizer.fit_transform(df_touse['bag_words'])
vectorized_bag_of_words = vectorized_bag_of_words.toarray()

In [17]:
def recommend(show_title, n_recom):
    similarity_matrix = cosine_similarity(vectorized_bag_of_words, vectorized_bag_of_words[list(np.where(df_touse["title"] == show_title)[0]), :])
    similarity_dataframe = pd.DataFrame(similarity_matrix)
    similarity_dataframe.index = title 
    similarity_dataframe =  similarity_dataframe.iloc[:,0]
    similarity_dataframe = similarity_dataframe.sort_values(ascending = False)
    similarity_dataframe = similarity_dataframe.drop_duplicates()
    return list(similarity_dataframe.index)[1:n_recom + 1]

In [18]:
recommend("Haikyuu!! Second Season",10)

['Haikyuu!!',
 'Haikyuu!!: Karasuno Koukou vs. Shiratorizawa Gakuen Koukou',
 'Ashita e Attack!',
 'Attacker You!',
 'Shoujo Fight: Norainu-tachi no Odekake',
 'Midoriyama Koukou Koushien-hen',
 'Dear Boys',
 'Diamond no Ace: Second Season',
 'Aoki Densetsu Shoot!',
 'Kuroko no Basket']

## Conclusion:
- At bare minimum, the first model will be the one used. Reduce dimensionality as much as possible
- Try doing PCA too later